# <center>Finding Stocks Based on Moving Averages</center>

## Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import datetime as dt
import yfinance as yf
import time as time

## Symbol Files

In [2]:
# # Importing Symbols File

# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

#### Importing Symbols

In [3]:
# Read the Excel data (NSE Stocks) into a DataFrame
df = pd.read_csv('All Yahoo Stock Symbols.csv', header=0)
df

,Symbol
0,EMCO.NS
1,HIRECT.NS
2,NIPPOBATRY.NS
3,ZENSARTECH.NS
4,FIBERWEB.NS
...,...
3105,BHAGERIA.BO
3106,GLOBE.BO
3107,RSYSTEMS.BO
3108,SALONA.BO


In [4]:
symbols=df['Symbol']

In [5]:
symbols.head()

0          EMCO.NS
1        HIRECT.NS
2    NIPPOBATRY.NS
3    ZENSARTECH.NS
4      FIBERWEB.NS
Name: Symbol, dtype: object

### No. of Days for moving average
- 50 vs 100
- 20 vs 50 (Golden crossover)

In [18]:
X=20
Y=50

### Defining Stock Data

In [19]:
# @title Span and Timeframe

interval="1d"

# interval can be 1m,1h,1d,1w,1m
# maximum of 7day data can be fetched from yahoo in single request if interval is of 1 minute

# Define the span of moving averages you want to check in days

span=14

# NOTE - To check last 14 days 100MA, atleast 114 rows of data is required 

### Start and End Date

In [21]:
## If you want to back test and find stocks doing crossover in past and not in the current time

end_date = '2023-05-01' # past date
end_date = datetime.strptime(end_date, '%Y-%m-%d')
end_date= end_date.date()
current_date=end_date

In [22]:
# Define Start and end date for present day crossovers

# current_date = dt.date.today()
# end_date = current_date.strftime("%Y-%m-%d")


In [23]:

if(interval=='1d' or interval=='1D'):
  x=(10+span+Y)*7/5  # 7/5 is done because we have 5 trading days in a week
  period= round(x,0)     # Define the time period in no. of Days
  print(period," Days")

if(interval=='1h' or interval=='1H'):
  x=(10+span+Y)*7/30  # 7/5 is done because we have 5 trading days in a week, and 6 hours in a trading day (5*6=30)
  period= round(x,0)     # Define the time period in no. of Days
  print(period," Days")


start_date = current_date-dt.timedelta(days=period)
start_date = start_date.strftime("%Y-%m-%d")

print("Start :",start_date)
print("End   :",end_date)


104.0  Days
Start : 2023-01-17
End   : 2023-05-01


# Finding Golden crossover stocks (SMA)


In [24]:
# @title Code

# Create an empty DataFrame to store the results
# signals_df = pd.DataFrame(columns=["Symbol", "Signal"])

# Create an empty list to store DataFrames for each symbol
data_frames = []

# Initialize counters
total_symbols = len(symbols)
symbols_processed = 0

# Measure the start time
start_time = time.time()

# Loop through each symbol
for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        # Extractinfo about the stock

        sector = info.get('sector', 'N/A')
        market_cap = info.get('marketCap', 'N/A')
        market_cap_sales = info.get('enterpriseToRevenue', 'N/A')
        promoter_holdings = info.get('heldPercentInsiders', 'N/A')
        pb_ratio = info.get('trailingPE', 'N/A')
        pe_ratio = info.get('forwardPE', 'N/A')

        market_cap = '{:.0f}'.format(market_cap / 10_000_000)
        promoter_holdings=promoter_holdings*100


        # Fetch historical data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date, interval=interval)

        # Calculate the 50-day and 100-day moving averages
        ma50 = stock_data['Close'].rolling(X).mean()
        ma100 = stock_data['Close'].rolling(Y).mean()
        last_close = stock_data['Close'][-1]

        # Get the first and last values of the moving averages
        first_ma50 = ma50[-span]
        last_ma50 = ma50[-1]
        first_ma100 = ma100[-span]
        last_ma100 = ma100[-1]

        # Calculate percentage changes
        pct_change_ma50_vs_ma100 = ((last_ma50 - last_ma100) / last_ma100) * 100
        pct_change_ma50_vs_close = ((last_close-last_ma50) / last_ma50) * 100


        # Check if the first value of ma50 is less than ma100 and the last value of ma50 is greater than ma100
        if first_ma50 < first_ma100 and last_ma50 > last_ma100:
            signal = 'Bullish'
        else:
            signal = 'None'

        # Create a DataFrame for the symbol and signal
        symbol_df = pd.DataFrame({
            "Symbol": [symbol],
            "Signal": [signal],
            "Price" : [last_close],
            "Market Cap." : [market_cap],
            "Sector" : [sector],
            "MarketCap/Sales": [market_cap_sales],
            "Promoter" : [promoter_holdings],
            "PB ratio" : [pb_ratio],
            "PE ratio" : [pe_ratio],
            "50MA" :[last_ma50],
            "100MA" :[last_ma100],
            "%chg. MA50_MA100": [pct_change_ma50_vs_ma100],
            "%chg. MA50_Close": [pct_change_ma50_vs_close]
        })

        # Append the DataFrame to the list
        data_frames.append(symbol_df)

    except Exception:
        # If there's an error, set the signal to "No data"
        symbol_df = pd.DataFrame({
            "Symbol": [symbol],
            "Signal": ["No data"],
            "Price" : [None],
            "Market Cap." : [None],
            "Sector" : [None],
            "MarketCap/Sales": [None],
            "Promoter" : [None],
            "PB ratio" : [None],
            "PE ratio" : [None],
            "50MA" :[None],
            "100MA" :[None],
            "%chg. MA50_MA100": [None],
            "%chg. MA50_Close": [None]
        })

        data_frames.append(symbol_df)

    # Update the progress counter
    symbols_processed += 1
    symbols_remaining = total_symbols - symbols_processed
    print(f"Processed {symbols_processed}/{total_symbols} symbols. {symbols_remaining} remaining.")

# Calculate the elapsed time
elapsed_time = (time.time() - start_time)/60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

# Concatenate all the DataFrames into one
Results = pd.concat(data_frames, ignore_index=True)


[*********************100%%**********************]  1 of 1 completed


Processed 1/3110 symbols. 3109 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 2/3110 symbols. 3108 remaining.


[*********************100%%**********************]  1 of 1 completed


...

Processed 3108/3110 symbols. 2 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 3109/3110 symbols. 1 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 3110/3110 symbols. 0 remaining.
Elapsed time: 20.58 minutes


## **Exporting Results**

In [25]:
# Export the DataFrame to a CSV file
csv_file_path = "Results/Results_MA.csv"  # Change the file path as needed
Results.to_csv(csv_file_path, index=False)
Results_MA=Results

In [26]:
value_counts = Results_MA['Signal'].value_counts()

# Print the frequency of values
print(value_counts)

None       1899
Bullish     875
No data     336
Name: Signal, dtype: int64


In [27]:
# # Filter symbols where the signal is "Bullish"
# bullish_signals_df = Results[Results['Signal'] == 'Bullish']

# # Print the resulting DataFrame
# print(bullish_signals_df)

In [28]:
Results_MA

,Symbol,Signal,Price,Market Cap.,Sector,MarketCap/Sales,Promoter,PB ratio,PE ratio,50MA,100MA,%chg. MA50_MA100,%chg. MA50_Close
0,EMCO.NS,None,1.95,13,N/A,N/A,N/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN...,N/A,N/A,1.95,1.95,0.0,0.0
1,HIRECT.NS,Bullish,214.0,1272,Industrials,2.752,71.779996,262.17316,N/A,205.844998,202.874998,1.463955,3.96172
2,NIPPOBATRY.NS,None,326.799988,473,Industrials,0.607,69.973,100.9547,N/A,320.0,325.127999,-1.577225,2.124996
3,ZENSARTECH.NS,None,278.25,14131,Technology,2.592,3.515,21.385172,N/A,274.127502,274.056001,0.02609,1.503861
4,FIBERWEB.NS,None,34.0,105,Consumer Cyclical,1.498,50.501996,N/A,N/A,32.805,34.225,-4.149014,3.642738
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105,BHAGERIA.BO,Bullish,131.850006,757,Basic Materials,1.498,81.383,43.775253,N/A,131.012499,130.363,0.498223,0.639258
3106,GLOBE.BO,No data,None,None,None,None,None,None,None,None,None,None,None
3107,RSYSTEMS.BO,None,252.699997,5425,Technology,3.275,82.509005,38.794415,N/A,256.222501,253.771,0.966029,-1.374783
3108,SALONA.BO,Bullish,250.5,156,Consumer Cyclical,0.492,78.179,18.697294,N/A,245.385001,235.935,4.005341,2.084479


# Finding Golden crossover stocks (EMA)

In [29]:
# @title Code

# Create an empty list to store DataFrames for each symbol
data_frames = []

# Initialize counters
total_symbols = len(symbols)
symbols_processed = 0

# Measure the start time
start_time = time.time()

# Loop through each symbol
for symbol in symbols:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        #Extracting info about the stock

        sector = info.get('sector', 'N/A')
        market_cap = info.get('marketCap', 'N/A')
        market_cap_sales = info.get('enterpriseToRevenue', 'N/A')
        promoter_holdings = info.get('heldPercentInsiders', 'N/A')
        pb_ratio = info.get('trailingPE', 'N/A')
        pe_ratio = info.get('forwardPE', 'N/A')

        market_cap = '{:.0f}'.format(market_cap / 10_000_000)
        promoter_holdings=promoter_holdings*100

        # Fetch historical data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date, interval=interval)

        # Calculate the 50-day and 100-day moving averages
        ema50 = stock_data['Close'].ewm(X).mean()
        ema100 = stock_data['Close'].ewm(Y).mean()
        last_close = stock_data['Close'][-1]

        # Get the first and last values of the moving averages
        first_ema50 = ema50[-span]
        last_ema50 = ema50[-1]
        first_ema100 = ema100[-span]
        last_ema100 = ema100[-1]

        # Calculate percentage changes
        pct_change_ema50_vs_ema100 = ((last_ema50 - last_ema100) / last_ema100) * 100
        pct_change_ema50_vs_close = ((last_close-last_ema50) / last_ema50) * 100


        # Check if the first value of ma50 is less than ma100 and the last value of ma50 is greater than ma100
        if first_ema50 < first_ema100 and last_ema50 > last_ema100:
            signal = 'Bullish'
        else:
            signal = 'None'

        # Create a DataFrame for the symbol and signal
        symbol_df = pd.DataFrame({
            "Symbol": [symbol],
            "Signal": [signal],
            "Price" : [last_close],
            "Market Cap." : [market_cap],
            "Sector" : [sector],
            "MarketCap/Sales": [market_cap_sales],
            "Promoter" : [promoter_holdings],
            "PB ratio" : [pb_ratio],
            "PE ratio" : [pe_ratio],
            "50EMA" :[last_ema50],
            "100EMA" :[last_ema100],
            "%chg. EMA50_EMA100": [pct_change_ema50_vs_ema100],
            "%chg. EMA50_Close": [pct_change_ema50_vs_close]
        })

        # Append the DataFrame to the list
        data_frames.append(symbol_df)

    except Exception:
        # If there's an error, set the signal to "No data"
        symbol_df = pd.DataFrame({
            "Symbol": [symbol],
            "Signal": ["No data"],
            "Price" : [last_close],
            "Market Cap." : [None],
            "Sector" : [None],
            "MarketCap/Sales": [None],
            "Promoter" : [None],
            "PB ratio" : [None],
            "PE ratio" : [None],
            "50EMA" :[None],
            "100EMA" :[None],
            "%chg. EMA50_EMA100": [None],
            "%chg. EMA50_Close": [None]
        })

        data_frames.append(symbol_df)

    # Update the progress counter
    symbols_processed += 1
    symbols_remaining = total_symbols - symbols_processed
    print(f"Processed {symbols_processed}/{total_symbols} symbols. {symbols_remaining} remaining.")

# Calculate the elapsed time
elapsed_time = (time.time() - start_time)/60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

# Concatenate all the DataFrames into one
Results = pd.concat(data_frames, ignore_index=True)


[*********************100%%**********************]  1 of 1 completed


Processed 1/3110 symbols. 3109 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 2/3110 symbols. 3108 remaining.


[*********************100%%**********************]  1 of 1 completed


...

Processed 3108/3110 symbols. 2 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 3109/3110 symbols. 1 remaining.


[*********************100%%**********************]  1 of 1 completed


Processed 3110/3110 symbols. 0 remaining.
Elapsed time: 20.26 minutes


## **Exporting Results**

In [30]:
# Export the DataFrame to a CSV file
csv_file_path = "Results/Results_EMA.csv"  # Change the file path as needed
Results.to_csv(csv_file_path, index=False)
Results_EMA=Results

In [31]:
value_counts = Results_EMA['Signal'].value_counts()

# Print the frequency of values
print(value_counts)

None       2226
Bullish     550
No data     334
Name: Signal, dtype: int64


In [32]:
# # Filter symbols where the signal is "Bullish"
# bullish_signals_df = Results[Results['Signal'] == 'Bullish']

# # Print the resulting DataFrame
# print(bullish_signals_df)

In [33]:
Results_EMA

,Symbol,Signal,Price,Market Cap.,Sector,MarketCap/Sales,Promoter,PB ratio,PE ratio,50EMA,100EMA,%chg. EMA50_EMA100,%chg. EMA50_Close
0,EMCO.NS,None,1.950000,13,N/A,N/A,N/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN/AN...,N/A,N/A,1.95,1.95,0.0,0.0
1,HIRECT.NS,None,214.000000,1272,Industrials,2.752,71.779996,262.17316,N/A,205.922718,205.955597,-0.015964,3.922482
2,NIPPOBATRY.NS,None,326.799988,477,Industrials,0.607,69.973,101.699036,N/A,325.06096,330.708425,-1.707687,0.534985
3,ZENSARTECH.NS,None,278.250000,14154,Technology,2.592,3.515,21.4195,N/A,271.607006,267.816035,1.415513,2.445811
4,FIBERWEB.NS,None,34.000000,105,Consumer Cyclical,1.498,50.501996,N/A,N/A,33.623175,34.468131,-2.451415,1.120731
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105,BHAGERIA.BO,None,131.850006,755,Basic Materials,1.498,81.383,43.67424,N/A,131.769945,132.276963,-0.3833,0.060758
3106,GLOBE.BO,No data,131.850006,None,None,None,None,None,None,None,None,None,None
3107,RSYSTEMS.BO,None,252.699997,5437,Technology,3.275,82.509005,38.88325,N/A,254.377863,252.63182,0.691141,-0.659596
3108,SALONA.BO,Bullish,250.500000,156,Consumer Cyclical,0.492,78.179,18.697294,N/A,241.229696,239.760087,0.61295,3.842937
